In [1]:
import sqlparse

from django.test import RequestFactory
from django.db import connection, reset_queries

from cast.models.repository import BlogIndexRepository

In [2]:
def count_queries(func):
    reset_queries()
    renditions = list(func())
    return len(connection.queries)


def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print(formatted_sql)
        

def blocker(*_args):
    """Get a traceback when a query is executed."""
    raise Exception("No database access allowed here.")

In [3]:
blog = Blog.objects.get(slug="ephes_blog")

In [4]:
rf = RequestFactory()
request = rf.get(blog.get_url())
request.htmx = False

In [5]:
%%time
reset_queries()
cachable_data = BlogIndexRepository.data_for_blog_index_cachable(request=request, blog=blog)

CPU times: user 41 ms, sys: 3.54 ms, total: 44.6 ms
Wall time: 57.7 ms


In [6]:
%%time
repository = BlogIndexRepository.create_from_cachable_data(data=cachable_data)

CPU times: user 8.81 ms, sys: 1.4 ms, total: 10.2 ms
Wall time: 8.95 ms


In [7]:
len(connection.queries)

17

In [8]:
# repository.template_base_dir = "bootstrap4"

In [9]:
%%time
reset_queries()
# with connection.execute_wrapper(blocker):
response = blog.serve(request, repository=repository).render()
print(len(connection.queries))

1
CPU times: user 64.2 ms, sys: 24.1 ms, total: 88.3 ms
Wall time: 88.2 ms


In [10]:
show_queries(connection.queries)

SELECT "cast_templatebasedirectory"."id",
       "cast_templatebasedirectory"."site_id",
       "cast_templatebasedirectory"."name"
FROM "cast_templatebasedirectory"
WHERE "cast_templatebasedirectory"."site_id" = 1
LIMIT 21


In [11]:
response.context_data

{'page': <Blog: Ephes Blog>,
 'self': <Blog: Ephes Blog>,
 'request': <WSGIRequest: GET '/blogs/ephes_blog/'>,
 'repository': <cast.models.repository.BlogIndexRepository at 0x1101b1c10>,
 'ellipsis': '…',
 'page_number': 1,
 'page_range': [1, 2, 3, '…', 38],
 'is_paginated': True,
 'has_previous': False,
 'previous_page_number': None,
 'has_next': True,
 'next_page_number': 2,
 'object_list': [<Post: Weeknotes 2024-04-22>,
  <Post: Weeknotes 2024-04-15>,
  <Post: Weeknotes 2024-04-08>,
  <Post: Weeknotes 2024-04-01>,
  <Post: Weeknotes 2024-03-25>],
 'filterset': <cast.filters.PostFilterset at 0x1101b11c0>,
 'template_base_dir': 'bootstrap5',
 'use_audio_player': False,
 'root_nav_links': [('/blogs/das_claas_und_nora_blog/',
   'Claas und Nora sagen Hallo'),
  ('/blogs/ephes_blog/', 'Ephes Blog')],
 'posts': [<Post: Weeknotes 2024-04-22>,
  <Post: Weeknotes 2024-04-15>,
  <Post: Weeknotes 2024-04-08>,
  <Post: Weeknotes 2024-04-01>,
  <Post: Weeknotes 2024-03-25>],
 'blog': <Blog: Ephe